In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt

# Transform
transform = transforms.Compose([transforms.ToTensor()])

# 전체 train/test 데이터 로딩
train_data = datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform)

# 0과 6 클래스만 필터링
def filter_dataset(dataset, classes=(0, 6)):
    indices = [i for i, (_, label) in enumerate(dataset) if label in classes]
    return Subset(dataset, indices)

train_filtered = filter_dataset(train_data)
test_filtered = filter_dataset(test_data)

In [ ]:
# 클래스별 분포 확인
from collections import Counter

def get_class_distribution(dataset):
    labels = [label for _, label in dataset]
    return Counter(labels)

print("Train 클래스 분포:", get_class_distribution(train_filtered))
print("Test 클래스 분포:", get_class_distribution(test_filtered))

In [ ]:
# 샘플 이미지 시각화
def show_samples(dataset, label, n=5):
    fig, axes = plt.subplots(1, n, figsize=(10, 2))
    count = 0
    for i in range(len(dataset)):
        img, lbl = dataset[i]
        if lbl == label:
            axes[count].imshow(img.squeeze(), cmap='gray')
            axes[count].set_title(f"Label: {lbl}")
            axes[count].axis('off')
            count += 1
        if count == n:
            break
    plt.show()

show_samples(train_filtered, label=0)
show_samples(train_filtered, label=6)

In [ ]:
# 클래스별 평균 이미지 비교
def compute_mean_image(dataset, label):
    images = [img.numpy() for img, lbl in dataset if lbl == label]
    return np.mean(images, axis=0).squeeze()

mean_img_0 = compute_mean_image(train_filtered, 0)
mean_img_6 = compute_mean_image(train_filtered, 6)

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(mean_img_0, cmap='viridis')
plt.title("Mean Image: Class 0")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(mean_img_6, cmap='viridis')
plt.title("Mean Image: Class 6")
plt.colorbar()
plt.show()

In [ ]:
# PCA로 차원 축소 후 시각화(QNN용 입력 준비 참고)
from sklearn.decomposition import PCA

def extract_images_and_labels(dataset):
    images = []
    labels = []
    for img, lbl in dataset:
        images.append(img.numpy().flatten())
        labels.append(lbl)
    return np.array(images), np.array(labels)

X_train, y_train = extract_images_and_labels(train_filtered)

# 정규화 (QNN angle encoding 고려 → [-π, π]로 정규화할 수도 있음)
X_train = X_train / 255.0

# PCA 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train)

plt.figure(figsize=(6, 5))
plt.scatter(X_pca[y_train == 0, 0], X_pca[y_train == 0, 1], alpha=0.5, label='Class 0')
plt.scatter(X_pca[y_train == 6, 0], X_pca[y_train == 6, 1], alpha=0.5, label='Class 6')
plt.title("PCA (2D) of Fashion-MNIST (0 vs 6)")
plt.legend()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# t-SNE 시각화 (수정된 버전)
X_tsne = TSNE(n_components=2, perplexity=30, max_iter=1000, random_state=42).fit_transform(X_train)

plt.figure(figsize=(6, 5))
plt.scatter(X_tsne[y_train == 0, 0], X_tsne[y_train == 0, 1], alpha=0.5, label='Class 0')
plt.scatter(X_tsne[y_train == 6, 0], X_tsne[y_train == 6, 1], alpha=0.5, label='Class 6')
plt.title("t-SNE of Fashion-MNIST (0 vs 6)")
plt.legend()
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.grid(True)
plt.show()

In [ ]:
# 필요 라이브러리
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import plotly.graph_objects as go
import numpy as np

# 데이터 불러오기 (Fashion-MNIST)
transform = transforms.ToTensor()
dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)

# 클래스 0과 6 추출 (각 300개)
indices_0 = (dataset.targets == 0).nonzero(as_tuple=True)[0]
indices_6 = (dataset.targets == 6).nonzero(as_tuple=True)[0]
selected_indices = torch.cat([indices_0, indices_6])
images = dataset.data[selected_indices].unsqueeze(1).float() / 255.0
labels = dataset.targets[selected_indices]

# 간단한 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(16 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 3)  # 3차원 특성 벡터
        )
    def forward(self, x):
        return self.net(x)

# 특징 추출
model = SimpleCNN()
model.eval()
with torch.no_grad():
    features = model(images)

features_np = features.numpy()
labels_np = labels.numpy()

# Plotly로 시각화
features_0 = features_np[labels_np == 0]
features_6 = features_np[labels_np == 6]

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=features_0[:, 0], y=features_0[:, 1], z=features_0[:, 2],
    mode='markers', marker=dict(size=3, color='blue'), name='Class 0'
))
fig.add_trace(go.Scatter3d(
    x=features_6[:, 0], y=features_6[:, 1], z=features_6[:, 2],
    mode='markers', marker=dict(size=3, color='orange'), name='Class 6'
))
fig.update_layout(
    title='3D CNN Feature Visualization (Class 0 vs 6)',
    scene=dict(xaxis_title='Feature 1', yaxis_title='Feature 2', zaxis_title='Feature 3'),
    margin=dict(l=0, r=0, b=0, t=40)
)
fig.show()